In [19]:
import numpy as np
import chandrasekhar_solver as ch
import Functions as F
from scipy.interpolate import UnivariateSpline
import matplotlib.pyplot as plt

def chandrasekhar_mass():
    
    ### DATA ### 
    M,logg = F.data_reader()
    M,R = F.scaler(M,logg)

    D = 1830000000 #found
    rho_c = [1e8,2e8,3e8,4e8,5e8,6e8,7e8,8e8,9e8,1e9,2e9,3e9,4e9,5e9,6e9,7e9,8e9,9e9,
             1e10,5e10,9e10,
            1e11,5e11, 1e12,5e12,1e13,5e13, 1e14, 1e15, 1e16, 1e17, 1e18, 1e19, 1e24, 1e25, 1e26,1e27]

    ### CONSTANTS(SI) ###
    K = 3097671.1345466143 #found before
    G = 6.67408e-11
    solar_mass = 1.988e30 #kg
    earth_radius = 6.371e6 #m

    zeta_n = 0 ## when the solution reaches surface value.
    zeta_prime_n = 0 ## when the derivative reaches the surface
    Radius = 0
    Mass = 0
    R_new = [] ## for storing new radius and mass
    M_new = []
    C = 5.165215436316856e+21 #found
    for rho in rho_c:
        ### solve the chandrasekhar for given rho and D ###
        sol,surface = ch.solve_chan(D,rho,limit=8.2,met='RK45')
        ##### find the surface #####
        for j in range(len(sol.t)):
            if (sol.y[0,j] <= surface):
                zeta_n = sol.t[j]
                zeta_prime_n = sol.y[1,j]
            elif(sol.y[0,-1]> surface):
                zeta_n = sol.t[-1]
                zeta_prime_n = sol.y[1,-1]
        ### For Radius ###
        y_c = np.sqrt(rho/D + 1)
        Beta = np.sqrt((2*C)/(np.pi*G))/(D*y_c) ## scale factor of radius
        Radius = (Beta*zeta_n)
        ### For Mass ###
        Mass = 4*np.pi*(Radius**3)*D*(y_c**3)*(-zeta_prime_n/zeta_n)
        Mass = Mass/solar_mass
        Radius = Radius/earth_radius
        ## add them to array
        R_new.append(Radius)
        M_new.append(Mass)
    R_new = np.array(R_new)
    M_new = np.array(M_new)
    fig, axis = plt.subplots(figsize = (9,5))
    plt.plot(R_new,M_new,'r--',R_new,M_new,'o')
    plt.title("mass-radius")
    plt.xlabel("Radius(earth radius)")
    plt.ylabel("Mass(solar mass)")
    print("given rho_c's : ", rho_c)
    print("and corresponding masses: ",M_new)
    print("difference between two last mass values: ", M_new[-1]-M_new[-2])
    print("Chandrasekhar Mass is : ",M_new[-1],"(in solar mass)")
